In [15]:
from gtts import gTTS
import os
import random, sys
from pydub import AudioSegment

COMB_ID = 0
ASCII_OFFSET = 399
EMPTY_PLACEHOLDER = "_"

# example text to speech
#myobj = gTTS(text='slip n go', lang="en", slow=False)
#myobj.save("welcome.mp3")
#os.system("welcome.mp3")

In [16]:
def create_combination(name, expected_time):
    global COMB_ID
    COMB_ID += 1
    return {
        "name": name,
        "time": expected_time,
        "id": COMB_ID
    }

combinations=[
    create_combination("up and around", 3),
    create_combination("slip and go", 4),
    create_combination("hook body", 4),
    create_combination("angles", 2),
    create_combination("double up", 4)
]

In [19]:
def create_round(time, gap, combinations):
    schedule = [EMPTY_PLACEHOLDER] * time
    second = gap
    while second < time:
        combination = random.choice(combinations)
        for i in range(second, min(second+combination["time"], time)):
            schedule[i] = chr(ASCII_OFFSET+combination["id"])
        second += combination["time"] + gap
    return schedule

In [21]:
"".join(create_round(180, 3, combinations))

'___ƓƓ___ƐƐƐ___ƔƔƔƔ___ƑƑƑƑ___ƑƑƑƑ___ƑƑƑƑ___ƐƐƐ___ƔƔƔƔ___ƑƑƑƑ___ƓƓ___ƓƓ___ƐƐƐ___ƔƔƔƔ___ƔƔƔƔ___ƒƒƒƒ___ƒƒƒƒ___ƔƔƔƔ___ƓƓ___ƒƒƒƒ___ƓƓ___ƒƒƒƒ___ƑƑƑƑ___ƑƑƑƑ___ƐƐƐ___ƐƐƐ___ƑƑƑƑ___ƑƑƑƑ___ƒƒƒ'

In [34]:
class Workout:

    def __init__(self, num_rounds, round_time, rest_time, comb_gap, combinations):
        self.workout = []
        self.round_time = round_time
        self.rest_time = rest_time
        self.comb_gap = comb_gap
        self.combinations = combinations
        self.output_fname = "output.mp3"
        for i in range(num_rounds):
            round_ = create_round(time=round_time, gap=comb_gap, combinations=combinations)
            self.workout = [*self.workout, *round_] + [EMPTY_PLACEHOLDER]*rest_time
        self.workout = self.workout[:-rest_time]
            
    def __iter__(self):
        for round in self.rounds:
            yield round
    
    def get_combination_by_id(self, id):
        for combination in self.combinations:
            if combination["id"] == id:
                return combination

    def add_sound(self, sound):
        previous_sound = AudioSegment.from_mp3(self.output_fname)
        (previous_sound + sound).export(self.output_fname, format="mp3")

    def create_mp3(self):
        flag = False
        for idx, second in enumerate(self.workout):
            if second == EMPTY_PLACEHOLDER:
                flag = False
                continue
            if flag: continue

            comb_id = ord(second) - ASCII_OFFSET

            combination = self.get_combination_by_id(comb_id)
            text = combination["name"]
            
            fname = "tmp"+str(comb_id)+".mp3"
            Speech(text, "en").save(fname)

            speech_sound = AudioSegment.from_mp3(fname) + 10
            silent_gap = AudioSegment.silent(self.comb_gap * 1000)
            
            latest_sound = silent_gap + speech_sound
            latest_sound += AudioSegment.silent((combination["time"]*1000 - len(speech_sound)))
            self.add_sound(latest_sound)

            flag = True
            
            if idx % self.round_time == 0:
                # adding bell sound after each round
                self.add_sound(AudioSegment.from_mp3("bell.mp3"))

In [32]:
"".join(workout.workout)

'___ƓƓ___ƔƔƔƔ___ƓƓ___ƒƒƒƒ___ƓƓ___ƐƐƐ___ƔƔƔƔ___ƒƒƒƒ___ƓƓ___ƑƑƑƑ___ƔƔƔƔ___ƓƓ___ƒƒƒƒ___ƐƐƐ___ƓƓ___ƓƓ___ƐƐƐ___ƑƑƑƑ___ƑƑƑƑ___ƓƓ___ƒƒƒƒ___ƐƐƐ___ƑƑƑƑ___ƒƒƒƒ___ƒƒƒƒ___ƓƓ___ƒƒƒƒ___ƓƓ___ƐƐƐ_________________________________________________________________ƒƒƒƒ___ƑƑƑƑ___ƑƑƑƑ___ƒƒƒƒ___ƓƓ___ƓƓ___ƒƒƒƒ___ƑƑƑƑ___ƔƔƔƔ___ƒƒƒƒ___ƑƑƑƑ___ƒƒƒƒ___ƒƒƒƒ___ƓƓ___ƒƒƒƒ___ƔƔƔƔ___ƐƐƐ___ƑƑƑƑ___ƒƒƒƒ___ƑƑƑƑ___ƑƑƑƑ___ƓƓ___ƔƔƔƔ___ƔƔƔƔ___ƐƐƐ___ƒƒƒƒ___ƓƓ__________________________________________________________________ƔƔƔƔ___ƔƔƔƔ___ƐƐƐ___ƑƑƑƑ___ƓƓ___ƒƒƒƒ___ƐƐƐ___ƑƑƑƑ___ƔƔƔƔ___ƒƒƒƒ___ƒƒƒƒ___ƔƔƔƔ___ƔƔƔƔ___ƓƓ___ƐƐƐ___ƑƑƑƑ___ƑƑƑƑ___ƔƔƔƔ___ƔƔƔƔ___ƒƒƒƒ___ƒƒƒƒ___ƐƐƐ___ƐƐƐ___ƑƑƑƑ___ƑƑƑƑ___ƓƓ___ƓƓ___Ɣ_______________________________________________________________ƔƔƔƔ___ƑƑƑƑ___ƓƓ___ƔƔƔƔ___ƓƓ___ƔƔƔƔ___ƑƑƑƑ___ƑƑƑƑ___ƔƔƔƔ___ƒƒƒƒ___ƓƓ___ƑƑƑƑ___ƔƔƔƔ___ƓƓ___ƑƑƑƑ___ƓƓ___ƔƔƔƔ___ƒƒƒƒ___ƓƓ___ƔƔƔƔ___ƐƐƐ___ƐƐƐ___ƐƐƐ___ƓƓ___ƑƑƑƑ___ƒƒƒƒ___ƔƔƔƔ___ƓƓ__________________________________________________________________ƔƔƔƔ___ƔƔƔƔ___ƔƔƔƔ___ƑƑƑƑ___ƐƐƐ___ƔƔ

In [31]:
workout = Workout(num_rounds=5, round_time=180, rest_time=60, comb_gap=3, combinations=combinations)
#for round in workout: print(round)

#workout.create_mp3()


In [ ]:
from pydub import AudioSegment
from google_speech import Speech


sound = AudioSegment.from_mp3("s.mp3")
print(len(sound))

sound = sound[1215000:]

print("exporting")
sound.export("cut.mp3", format="mp3")
